In [2]:
# Dependencies# Depen 
import requests
import json
import pandas as pd
# Google developer API key. Create a config.py and put your own google api key to the file.
from config import gkey
import numpy as np

#### Filling out the empty county cells using Google API

In [3]:
# Import the raw data
drug_death_raw_df = pd.read_csv("Resources/Accidental_Drug_Related_Deaths_2012-2017.csv")
drug_death_raw_df = drug_death_raw_df[1:]
drug_death_raw_df.columns

Index(['CaseNumber', 'Date', 'Year', 'Sex', 'Race', 'Age', 'Residence City',
       'Residence State', 'Residence County', 'Death City', 'Death State',
       'Death County', 'Location', 'DescriptionofInjury', 'InjuryPlace',
       'ImmediateCauseA', 'Heroin', 'Cocaine', 'Fentanyl', 'Oxycodone',
       'Oxymorphone', 'EtOH', 'Hydrocodone', 'Benzodiazepine', 'Methadone',
       'Amphet', 'Tramad', 'Morphine (not heroin)', 'Other', 'Any Opioid',
       'MannerofDeath', 'AmendedMannerofDeath', 'DeathLoc',
       'DeathLocationCity', 'DeathLocLat', 'DeathLocLong'],
      dtype='object')

In [4]:
# index = 0
for rindex, row in drug_death_raw_df.iterrows():
    death_county = row['Death County']

    if pd.isnull(death_county):
        
        # Use Google geo-api to populate the empty county
        death_city = row['DeathLocationCity']

        # Target city
        target_city = f"{death_city}, CT"

        # Build the endpoint URL
        target_url = "https://maps.googleapis.com/maps/api/geocode/json?" \
            "address=%s&key=%s" % (target_city, gkey)


        try:
            geo_data = requests.get(target_url).json()

            county_name = ' '.join(geo_data["results"][0]["address_components"][1]['long_name'].split(' ')[:2])

            # Populate the death county cell
            drug_death_raw_df.loc[rindex,'Death County'] = county_name
            
        except:
            print("Failed.")

#### Cleaning the county names

In [5]:
drug_death_raw_df['Death County'].value_counts()

HARTFORD              1022
NEW HAVEN              975
FAIRFIELD              555
NEW LONDON             319
LITCHFIELD             187
Hartford County        181
MIDDLESEX              154
New Haven              140
WINDHAM                116
TOLLAND                100
Fairfield County        99
New London              37
Manchester              29
Windham County          23
Litchfield              16
Wallingford             15
Tolland County          14
Litchfield County       11
Windham                 10
Middlesex County        10
Huntington               8
Colchester               7
Portland                 6
Groton                   6
Guilford                 4
Deep River               4
Montville                4
Waterford                3
Stonington               3
Killingly                2
Simsbury                 2
Newtown                  2
Bethel                   2
East Windsor             2
Greenwich                1
USA                      1
Connecticut Avenue       1
W

In [6]:
# County names in uppercase
drug_death_raw_df['Death County'] = drug_death_raw_df['Death County'].str.upper()
# Remove 'County' from the county names
remove_county = lambda x: ' '.join(x.split(' ')[:-1]) if 'COUNTY' in x else x
drug_death_raw_df['Death County'] = drug_death_raw_df['Death County'].apply(remove_county)
# Remove whitespace
drug_death_raw_df['Death County'] = drug_death_raw_df['Death County'].str.strip()
# Clean the county cells filled with city names
cities = {'MANCHESTER':'HARTFORD', 'WALLINGFORD':'NEW HAVEN',
         'HUNTINGTON':'FAIRFIELD','COLCHESTER':'NEW LONDON',
         'PORTLAND':'MIDDLESEX', 'GROTON':'NEW LONDON',
         'GUILFORD':'NEW HAVEN', 'MONTVILLE':'NEW LONDON',
         'DEEP RIVER':'MIDDLESEX','STONINGTON':'NEW LONDON',
         'WATERFORD':'NEW LONDON','KILLINGLY':'WINDHAM',
         'SIMSBURY':'HARTFORD', 'EAST WINDSOR':'HARTFORD',
         'NEWTOWN':'FAIRFIELD', 'BETHEL':'FAIRFIELD',
         'WESTBROOK':'MIDDLESEX','HADDAM':'MIDDLESEX',
         'WINCHESTER':'LITCHFIELD','SPRAGUE':'NEW LONDON',
         'GREENWICH':'FAIRFIELD','MADISON':'NEW HAVEN',
         'WATERTOWN':'LITCHFIELD','GRISWOLD':'NEW LONDON',
         'PLYMOUTH':'LITCHFIELD','MANSFIELD':'TOLLAND',
         'CONNECTICUT AVENUE':'HARTFORD', 'SUFFIELD':'HARTFORD'}
replace_cities = lambda x: cities.get(x) if x in cities.keys() else x
drug_death_raw_df['Death County']= drug_death_raw_df['Death County'].apply(replace_cities)

In [7]:
drug_death_df = drug_death_raw_df[drug_death_raw_df['Death County'] != 'USA']

In [8]:
drug_death_df['Death County'].value_counts()

HARTFORD      1238
NEW HAVEN     1135
FAIRFIELD      668
NEW LONDON     381
LITCHFIELD     217
MIDDLESEX      176
WINDHAM        151
TOLLAND        115
Name: Death County, dtype: int64

In [32]:
# Inspect the dataset
drug_death_df.count()

Unnamed: 0               4081
CaseNumber               4081
Date                     4079
Year                     4081
Sex                      4078
Race                     4071
Age                      4079
Residence City           3947
Residence State          2116
Residence County         3331
Death City               4078
Death State              2201
Death County             4081
Location                 4060
DescriptionofInjury      1495
InjuryPlace              4003
ImmediateCauseA          4081
Heroin                   2154
Cocaine                  1175
Fentanyl                 1466
Oxycodone                 547
Oxymorphone                97
EtOH                     1005
Hydrocodone               105
Benzodiazepine           1077
Methadone                 387
Amphet                    103
Tramad                     90
Morphine (not heroin)      56
Other                     378
Any Opioid                714
MannerofDeath            4074
AmendedMannerofDeath       28
DeathLoc  

#### Dropping the cases without date 

In [33]:
drug_death_df = drug_death_df[~pd.isna(drug_death_df['Date'])]
drug_death_df.count()

Unnamed: 0               4079
CaseNumber               4079
Date                     4079
Year                     4079
Sex                      4077
Race                     4070
Age                      4078
Residence City           3947
Residence State          2116
Residence County         3331
Death City               4078
Death State              2201
Death County             4079
Location                 4060
DescriptionofInjury      1495
InjuryPlace              4001
ImmediateCauseA          4079
Heroin                   2154
Cocaine                  1174
Fentanyl                 1466
Oxycodone                 547
Oxymorphone                97
EtOH                     1005
Hydrocodone               105
Benzodiazepine           1077
Methadone                 387
Amphet                    103
Tramad                     90
Morphine (not heroin)      56
Other                     377
Any Opioid                714
MannerofDeath            4073
AmendedMannerofDeath       28
DeathLoc  

In [40]:
# Populate the missing death state with 'CT'
drug_death_df['Death State'] = 'CT'

In [41]:
drug_death_df['Death State'].value_counts()

CT    4079
Name: Death State, dtype: int64

#### Removing the unnamed column

In [44]:
# Remove the unnamed columns
drug_death_df = drug_death_df.loc[:, ~drug_death_df.columns.str.contains('^Unnamed')]

,CaseNumber,Date,Year,Sex,Race,Age,Residence City,Residence State,Residence County,Death City,...,Tramad,Morphine (not heroin),Other,Any Opioid,MannerofDeath,AmendedMannerofDeath,DeathLoc,DeathLocationCity,DeathLocLat,DeathLocLong
0,12-16897,11/30/2012,2012,Male,White,45.0,NaN,NaN,NaN,NEW HAVEN,...,NaN,NaN,NaN,NaN,Accident,NaN,"NEW HAVEN, CT\r\r\n(41.308252, -72.924161)",NEW HAVEN,41.30825,72.924161
1,13-11849,8/12/2013,2013,Male,White,30.0,NEW HAVEN,NaN,NaN,NEW HAVEN,...,NaN,NaN,NaN,NaN,Accident,NaN,"NEW HAVEN, CT\r\r\n(41.308252, -72.924161)",NEW HAVEN,41.30825,72.924161
2,14-17578,11/23/2014,2014,Male,White,27.0,NAUGATUCK,NaN,NaN,NEW MILFORD,...,NaN,NaN,NaN,NaN,Accident,NaN,"NEW MILFORD, CT\r\r\n(41.576633, -73.408713)",NEW MILFORD,41.57663,73.408713
3,12-11497,8/14/2012,2012,Male,White,21.0,ENFIELD,NaN,NaN,ENFIELD,...,NaN,NaN,NaN,NaN,Accident,NaN,"ENFIELD, CT\r\r\n(41.976501, -72.591985)",ENFIELD,41.9765,72.591985
4,13-2133,2/4/2013,2013,Male,White,25.0,POMFRET,NaN,WINDHAM,POMFRET,...,NaN,NaN,NaN,NaN,Accident,NaN,"POMFRET, CT\r\r\n(41.897524, -71.962064)",POMFRET,41.89752,71.962064


#### Exporting to a CSV file 

In [47]:
drug_death_df.to_csv("Resources/Accidental_Drug_Related_Deaths_2012-2017_cleaned.csv",index=False)